In [4]:
""" 
json.dumps(): 将 Python 对象编码成 JSON 字符串。 json.dump(): 将 Python 对象编码成 JSON 字符串，并写入到文件中。json.loads(): 将已编码的 JSON 字符串解码为 Python 对象。json.load(): 读取文件中的 JSON 编码的数据，并解码为 Python 对象。
"""
import os
import json
import torch
torch.cuda.is_available()
def json_standard(input_object):

    # print(input_object,type(input_object))
    return json.dumps(input_object,ensure_ascii=False,indent=4)
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def check_file(file_path:str):
    """返回该路径下是否存在文件"""
    is_exist=os.path.exists(file_path)
    if is_exist:
        file_name=file_path.split("/")[-1]
        print(f"{file_name} exist")
    return is_exist

import torch
import json

from transformers import (AutoTokenizer)
model_id="/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2"
tokenizer=AutoTokenizer.from_pretrained(model_id)
# 这里不设置 trust_remote_code也行
tokenizer.padding_size='right'#填充符的位置


In [7]:
cn_data=read_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/cn_stra_labels.json")

In [8]:
# en_data[0]["topic_dialog"]["dialog"]
import copy
cn_data_backup=read_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/cn_stra_labels.json")

In [9]:
# en_data[4]['topic_dialog']['dialog'][3]
emo_label_few_shots=[
    {"label": "Question", "example": "Can you talk more about your feelings at that time?"},
    {"label": "Restatement or Paraphrasing", "example": "It sounds that you feel like everyone is ignoring you. Is it correct?"},
    {"label": "Reflection of Feelings", "example": "I understand how anxious you are."},
    {"label": "Self-disclosure", "example": "I feel the same way! I also don't know what to say to strangers."},
    {"label": "Affirmation and Reassurance", "example": "You've done your best and I believe you will get it!"},
    {"label": "Providing Suggestions", "example": "Deep breaths can help people calm down. Could you try to take a few deep breaths?"},
    {"label": "Information", "example": "Apparently, lots of research has found that getting enough sleep before an exam can help students perform better."},
    {"label": "Others", "example": "Yeah, I am glad to help you."}
    ]
[item['label'] for item in emo_label_few_shots]

['Question',
 'Restatement or Paraphrasing',
 'Reflection of Feelings',
 'Self-disclosure',
 'Affirmation and Reassurance',
 'Providing Suggestions',
 'Information',
 'Others']

In [10]:
from openai import OpenAI
import os
import time
import copy
import json
client = OpenAI()
client.base_url='https://gptnb.keqichen.top/v1'
result=[]
def get_strategy_messages(dialog_history:list):

  # emo_label_few_shot_str=json.dumps(emo_label_few_shot)

  example_list=[{"role":"user","content":"[...{'role':'Therapist','content':'One of your underlying assumptions is ‘If I don’t have a real job then I’m not as good as others’. Now you’ve been out of work for several years due to illness and you said you try to cram a lot of activity into each day to prove that you are as good as others by having a job at home, so to speak; so do you see reading the newspaper or watching television as an indulgence, idling when you should be striving?'}]"},{"role":"assistant","content":"{'strategy':[\'Reflection of Feelings\', \'Self-disclosure\',\'Question\']}"}]
  for idx,item in enumerate(emo_label_few_shots):
    example_list.append({"role":"user","content":"[...{'role':'Therapist','content':'"+item['example']+"'}]"})
    return_str_json={"strategy":f"['{item['label']}']"}
    example_list.append({"role":"assistant","content":json.dumps(return_str_json)})
  messages=[
    {"role": "system", "content": f" Please classify the strategy of the last utterance from the therapist. There are some examples:"},
  ]
  messages.extend(example_list)
  gpt_history=[]
  for dialog_item in dialog_history:
    if '"治疗师"' in dialog_item['participant']:
      gpt_history.append({"role": "Therapist", "content":dialog_item['text'],"reasoning":dialog_item['reasoning']})
    else:
      gpt_history.append({"role": "Client", "content":dialog_item['text']})
  messages.extend([
    {"role": "user","content": f"Yes, there is the dialog history.{str(gpt_history)} Please classify the strategy of the last utterance from the therapist and respond with a list of labels in JSON format. The possible strategies fall within these labels: ['Question', 'Restatement or Paraphrasing', 'Reflection of Feelings', 'Self-disclosure', 'Affirmation and Reassurance', 'Providing Suggestions', 'Information', 'Others', 'Unknown']."}
    ]
  )
  return messages
  
token_sum=0
def get_price(total_token:int):
  return f"GPT-4:{token_sum/1000000*10}$ GPT-3:{token_sum/1000000*0.5}$"
for x,topic in enumerate(cn_data):
  dialog_list=cn_data[x]['topic_dialog']['dialog']
  for y in range(0,len(dialog_list)-1):
    
    dialog_history=copy.deepcopy(dialog_list[max(y-4,0):y+1])
    # 窗口大小为 5 y+1-y+4=5
    # print(dialog_history)
    if 'strategy' in dialog_history[-1].keys():
      continue
    if '治疗师' in dialog_history[-1]['participant']:
      # 是'Therapist' 所以才需要预测~ strategy
      gpt_4="gpt-4-0613"
      gpt_3_5_model="gpt-3.5-turbo"
      print(dialog_history)
      emo_messages=get_strategy_messages(dialog_history)
      # print(emo_messages)
      completion = client.chat.completions.create(
        model=gpt_3_5_model,
        messages=emo_messages
      )
      token_sum+=completion.usage.total_tokens
      choice=completion.choices[0]
      time.sleep(0.5)
      print(token_sum,get_price(token_sum),dialog_history[-1]['text'])
      print(token_sum,choice)
      cn_data[x]['topic_dialog']['dialog'][y]['strategy']=choice.message.content
      print(cn_data[x]['topic_dialog']['dialog'][y]['strategy'],cn_data[x]['topic_dialog']['dialog'][y]['participant'])
      write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/cn_stra_labels.json",cn_data,True)
  print(token_sum,"\n\n\n\n")
    

0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 




0 





In [15]:
cn_data=read_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/cn_stra_labels.json")

[{'text': '你已经描述了你和丈夫之间关系的现状。现在描述一下你理想中的婚姻关系吧。嗯，换个方式问这个问题，有什么是你想要从他那里获得最终却并没有得到的？',
  'participant': '治疗师',
  'id': '11-0',
  'strategy': '{"strategy": ["Question"]}'}]

In [21]:
# en_data[4]['topic_dialog']['dialog'][3]
"""
Anxiety 281 26.7%
Depression 276 26.2%
Sadness 250 23.7%
Anger 96 9.1%
Fear 88 8.4%
Disgust 32 3.0%
Shame
"""
emotional_label_few_shots=[
{"label": "Anxiety", "example": "Yes, I am. When I don't have to relate everything to childhood sexual conflicts, I can listen better. / I’m walking down the high street. I feel very uncomfortable. I think everyone is looking at me, so I keep my head down. / I see them staring at me, judging me. I hate it."},
{"label": "Depression", "example": "But I didn't stay angry long at Nancy when she left me. I got depressed."},
{"label": "Sadness", "example": "I’m walking down the high street. I feel very uncomfortable. I think everyone is looking at me, so I keep my head down."},
{"label": "Anger", "example": "I never told you about my grandmother, did I?"},
{"label": "Fear", "example": "I'd say the only difference is that this time you'd be in the role of apprentice rather than teacher, and that more than sex per se is what frightens you..."},
{"label": "Disgust", "example": "I don't know who the people were. All I can remember is that there was some woman in it. She was kissing me, and it was nice until she turned into a man."},
{"label": "Guilty", "example": "I didn’t bring her up right. It’s my fault that she behaves as she does. If I’d been a good mother she wouldn’t have turned out like this. I’m a bad mother."},
{"label": "Shame", "example": "And the family I live with doesn’t know that."},
{"label": "Neutral", "example": "It doesn't really matter to me either way."},
{"label": "Unknown", "example": "Right."},
{"label": "Others", "example": "#Those utterances that don't belong to the classification above.#"},
]
[item['label'] for item in emotional_label_few_shots]

['Anxiety',
 'Depression',
 'Sadness',
 'Anger',
 'Fear',
 'Disgust',
 'Guilty',
 'Shame',
 'Neutral',
 'Unknown',
 'Others']

In [22]:
from openai import OpenAI
import os
import time
import copy
import json
client = OpenAI()
client.base_url='https://gptnb.keqichen.top/v1'
result=[]
def get_emotional_labels_messages(dialog_history:list):

  # emo_label_few_shot_str=json.dumps(emo_label_few_shot)
  example_list=[]
  for idx,item in enumerate(emotional_label_few_shots):
    example_list.append({"role":"user","content":"[...{'role':'Client','content':'"+item['example']+"'}]"})
    return_str_json={"emotional label":f"['{item['label']}']"}
    example_list.append({"role":"assistant","content":json.dumps(return_str_json)})
  messages=[
    {"role": "system", "content": f" Please classify the emotional label of the last utterance from the therapist. There are some examples:"},
  ]
  messages.extend(example_list)
  gpt_history=[]
  for dialog_item in dialog_history:
    if '治疗师' in dialog_item['participant']:
      gpt_history.append({"role": "Therapist", "content":dialog_item['text']})
    else:
      # gpt_history.append({"role": "Client", "content":dialog_item['text'],"observation":dialog_item['observation']})
      gpt_history.append({"role": "Client", "content":dialog_item['text']})
  messages.extend([
    {"role": "user","content": f"Yes, there is the dialog history. {str(gpt_history)} Please classify the emotional label of the last utterance from the Client, referring to the conversation history. Please respond with a list of labels in JSON format. The possible emotional labels fall within these labels: ['Anxiety','Depression','Sadness','Anger','Fear','Disgust','Guilty','Shame','Neutral','Unknown','Others']."}
    ]
  )
  return messages
  
token_sum=0
def get_price(total_token:int):
  return f"GPT-4:{token_sum/1000000*10}$ GPT-3:{token_sum/1000000*0.5}$"

for x,topic in enumerate(cn_data):
  dialog_list=cn_data[x]['topic_dialog']['dialog']
  for y in range(0,len(dialog_list)-1):
    
    dialog_history=copy.deepcopy(dialog_list[max(y-4,0):y+1])
    # 窗口大小为 5 y+1-y+4=5
    # print(dialog_history)
    if '治疗师' not in dialog_history[-1]['participant'] and "emotional label" not in dialog_history[-1].keys():
      # 是'Therapist' 所以才需要预测~ emotional label
      gpt_4="gpt-4-0613"
      gpt_3_5_model="gpt-3.5-turbo"
      emo_messages=get_emotional_labels_messages(dialog_history[-1:])
      completion = client.chat.completions.create(
        model=gpt_3_5_model,
        messages=emo_messages
      )
      token_sum+=completion.usage.total_tokens
      choice=completion.choices[0]
      time.sleep(0.5)
      print(x,y,token_sum,get_price(token_sum),dialog_history[-1]['text'])
      
      cn_data[x]['topic_dialog']['dialog'][y]['emotional label']=choice.message.content
      print(x,get_price(token_sum),cn_data[x]['topic_dialog']['dialog'][y]['emotional label'])
      write_file("/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/en_data_emo_stra_labels__6_9.json",cn_data,True)
  print(token_sum,"\n\n\n\n")
    

0 1 731 GPT-4:0.00731$ GPT-3:0.0003655$ 是的，我想我可以。
0 GPT-4:0.00731$ GPT-3:0.0003655$ {"emotional label": "Unknown"}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/en_data_emo_stra_labels__6_9.json
0 3 1470 GPT-4:0.0147$ GPT-3:0.000735$ 我希望看到十岁时的自己。
0 GPT-4:0.0147$ GPT-3:0.000735$ {"emotional label": "['Neutral']"}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/en_data_emo_stra_labels__6_9.json
1470 




1 1 2209 GPT-4:0.02209$ GPT-3:0.0011045$ 也许我应该再减一次肥。
1 GPT-4:0.02209$ GPT-3:0.0011045$ {"emotional label": "['Anxiety']"}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/en_data_emo_stra_labels__6_9.json
1 3 2934 GPT-4:0.029339999999999998$ GPT-3:0.001467$ 当然。
1 GPT-4:0.029339999999999998$ GPT-3:0.001467$ {"emotional label": ["Others"]}
写入/home/ckqsudo/code2024/EMNLP2024/Chinese_data/cache/en_data_emo_stra_labels__6_9.json
1 5 3664 GPT-4:0.03664$ GPT-3:0.001832$ 关于什么？
1 GPT-4:0.03664$ GPT-3:0.001832$ {"emotional label": "['Unknown']"}
写入/home/ckqsudo/code2024/EMNLP202